In [3]:
import operator
import random
import os,sys,shutil, glob, copy

import numpy
import pandas
import math
import matplotlib.pyplot as plt

import numpy as np

In [4]:
# x, y, Delta x, Delta y, concentration

pmin = [174875,532725,20,20, 10]
pmax = [174975,532825,50,50, 1]
smin = [1, 1,0.5,0.5,1.0e-9]
smax = [2.5, 2.5,1.0,1.0, 2.0e-9]


print(random.uniform(pmin[0], pmax[0]))

174907.61667199133


In [5]:
def evalHGS(part, obs):
    # copy direcotry
    dest_dir = './test'
    shutil.copytree('./Base',dest_dir,dirs_exist_ok=True)
    
    # write particle infromat
    lines = '''clear chosen nodes
choose nodes all

Initial concentration from output file
./05_initial/IC_sso.concentration.0266

clear chosen nodes
Choose nodes block
{0} {1}
{2} {3}
-10 0

Specified mass flux
1
157766400 504921600 {4}

Interpolate mass flux
clear chosen nodes

!----------------------- Time step control
transport time weighting
1.0

concentration control
1e-1

upstream weighting of velocities
1.0 1.0 1.0
!----------------------- Transport properties
use domain type
porous media

clear chosen zones
choose zones all
longitudinal dispersivity
15
transverse dispersivity
2.5
vertical transverse dispersivity
1.5
tortuosity
1.0

    '''.format(part[0]-part[2], part[0]+part[2], part[1]-part[3],part[1]+part[3], part[4])
    
    fname = os.path.join(dest_dir,'01_inc/concentration.inc')
    with open(fname,'w') as fid:
        fid.write(lines)
        
    # Run HGS.
    command = 'cd %s && grok.x > nul && hgs.x > nul '%(dest_dir)
    os.system(command)
    
    # evaluate misfit between data.
    weln = [obs[k]['name'] for k in obs]
    
    cost = 0.0
    for i, k in enumerate(obs):
        fname = os.path.join(dest_dir,'*conc.%s.*dat'%(obs[k]['name']))
        fname = glob.glob(fname)[0]
        point = np.loadtxt(fname,skiprows=3)
        # file_list = glob.glob(fname)
        # if not file_list:
        #     print(f"No file found for {obs[k]['name']} in {dest_dir}")
        #     continue
        # fname = file_list[0]

        # searching specific time?
        time = point[:,0]
        pos  = np.zeros(np.shape(time),dtype=bool)
        for j, t in enumerate(time):
            if np.any(t == np.array(obs[k]['time'])):
                pos[j] = True

        cost += np.sum((np.array(obs[k]['conc']) - point[pos,1])**2)
    return cost

In [6]:
weln = ['BH-1','BH-2','BH-3']
# print(prefix)
time_spc = np.array([94608000,110332800,126230400,139190400,157766400])

#fig, ax = plt.subplots()
obs = {}
for i, p in enumerate(weln):
    obs[i] = {'name':p, 'time':[],'conc':[]}
    fname = './*conc.%s.*.dat'%(p)
    print('Searching for file:', fname)
    fname = glob.glob(fname)[0]
    print(fname)
    point = np.loadtxt(fname,skiprows=3)
    
    # searching specific time?
    time = point[:,0]
    pos  = np.zeros(np.shape(time),dtype=bool)
    for j, t in enumerate(time):
        if np.any(t == time_spc):
            pos[j] = True
    obs[i]['time'] = point[pos,0]
    obs[i]['conc'] = point[pos,1]

print(obs)

Searching for file: ./*conc.BH-1.*.dat
.\IC_sso.observation_well_conc.BH-1.PCE.dat
Searching for file: ./*conc.BH-2.*.dat
.\IC_sso.observation_well_conc.BH-2.PCE.dat
Searching for file: ./*conc.BH-3.*.dat
.\IC_sso.observation_well_conc.BH-3.PCE.dat
{0: {'name': 'BH-1', 'time': array([9.460800e+07, 1.103328e+08, 1.262304e+08, 1.391904e+08,
       1.577664e+08]), 'conc': array([4.56632513e-07, 1.16604210e-06, 2.33270889e-06, 3.82950336e-06,
       7.45570947e-06])}, 1: {'name': 'BH-2', 'time': array([9.460800e+07, 1.103328e+08, 1.262304e+08, 1.391904e+08,
       1.577664e+08]), 'conc': array([6.64707517e-05, 1.75146972e-04, 3.77988072e-04, 6.57164441e-04,
       1.39223464e-03])}, 2: {'name': 'BH-3', 'time': array([9.460800e+07, 1.103328e+08, 1.262304e+08, 1.391904e+08,
       1.577664e+08]), 'conc': array([7.34908576e-09, 3.39507427e-08, 1.19949935e-07, 3.04395507e-07,
       1.12876743e-06])}}


In [7]:
import copy
import random
import numpy as np

# Initial parameters
T0 = 10
alpha = 0.9
iter_max = 100

# Cost function: same as evaluation function defined above
cost_func = evalHGS

# Initial solution
p = np.random.uniform(low=pmin, high=pmax)

# Initial cost
cost = cost_func(p, obs)

# Keep track of best solution
best_solution = p
best_cost = cost

# Simulated Annealing
for i in range(iter_max):
    for j in range(iter_max):
        # Generate a new solution
        p_new = p + np.random.uniform(low=smin, high=smax)
        p_new = np.maximum(p_new, pmin)
        p_new = np.minimum(p_new, pmax)

        # Calculate new cost
        cost_new = cost_func( p_new, obs)

        # If new solution is better, accept it
        if cost_new < cost:
            p = p_new
            cost = cost_new

            # Update best solution
            if cost_new < best_cost:
                best_solution = p_new
                best_cost = cost_new

        # If new solution is worse, accept it with a probability
        elif random.random() < np.exp((cost - cost_new) / T0):
            p = p_new
            cost = cost_new

    # Decrease the temperature
    T0 *= alpha

print("Best solution:", best_solution)
print("Best cost:", best_cost)

print(p)

Error: [('./Base\\hs.dbg', './test\\hs.dbg', "[Errno 13] Permission denied: './test\\\\hs.dbg'"), ('./Base\\IC_sso.cen', './test\\IC_sso.cen', "[Errno 13] Permission denied: './test\\\\IC_sso.cen'"), ('./Base\\IC_sso.hen', './test\\IC_sso.hen', "[Errno 13] Permission denied: './test\\\\IC_sso.hen'"), ('./Base\\IC_sso.lst', './test\\IC_sso.lst', "[Errno 13] Permission denied: './test\\\\IC_sso.lst'"), ('./Base\\IC_sso.mass_balance_cumulative.PCE.dat', './test\\IC_sso.mass_balance_cumulative.PCE.dat', "[Errno 13] Permission denied: './test\\\\IC_sso.mass_balance_cumulative.PCE.dat'"), ('./Base\\IC_sso.mass_balance_raw.PCE.dat', './test\\IC_sso.mass_balance_raw.PCE.dat', "[Errno 13] Permission denied: './test\\\\IC_sso.mass_balance_raw.PCE.dat'"), ('./Base\\IC_sso.mass_balance_summary.PCE.dat', './test\\IC_sso.mass_balance_summary.PCE.dat', "[Errno 13] Permission denied: './test\\\\IC_sso.mass_balance_summary.PCE.dat'"), ('./Base\\IC_sso.observation_well_conc.BH-1.PCE.dat', './test\\IC_ss

: 

In [1]:
import numpy as np
import shutil
import os
import glob


import os,sys,shutil, glob, copy

import matplotlib.pyplot as plt

import numpy as np


In [2]:

# objective function
def objective(x):
	return evalHGS(x, obs)


In [3]:

weln = ['BH-1','BH-2','BH-3']
# print(prefix)
time_spc = np.array([94608000,110332800,126230400,139190400,157766400])

#fig, ax = plt.subplots()
obs = {}
for i, p in enumerate(weln):
    obs[i] = {'name':p, 'time':[],'conc':[]}
    fname = './*conc.%s.*.dat'%(p)
    print('Searching for file:', fname)
    fname = glob.glob(fname)[0]
    print(fname)
    point = np.loadtxt(fname,skiprows=3)
    
    # searching specific time?
    time = point[:,0]
    pos  = np.zeros(np.shape(time),dtype=bool)
    for j, t in enumerate(time):
        if np.any(t == time_spc):
            pos[j] = True
    obs[i]['time'] = point[pos,0]
    obs[i]['conc'] = point[pos,1]

print(obs)


Searching for file: ./*conc.BH-1.*.dat
.\IC_sso.observation_well_conc.BH-1.PCE.dat
Searching for file: ./*conc.BH-2.*.dat
.\IC_sso.observation_well_conc.BH-2.PCE.dat
Searching for file: ./*conc.BH-3.*.dat
.\IC_sso.observation_well_conc.BH-3.PCE.dat
{0: {'name': 'BH-1', 'time': array([9.460800e+07, 1.103328e+08, 1.262304e+08, 1.391904e+08,
       1.577664e+08]), 'conc': array([4.56632513e-07, 1.16604210e-06, 2.33270889e-06, 3.82950336e-06,
       7.45570947e-06])}, 1: {'name': 'BH-2', 'time': array([9.460800e+07, 1.103328e+08, 1.262304e+08, 1.391904e+08,
       1.577664e+08]), 'conc': array([6.64707517e-05, 1.75146972e-04, 3.77988072e-04, 6.57164441e-04,
       1.39223464e-03])}, 2: {'name': 'BH-3', 'time': array([9.460800e+07, 1.103328e+08, 1.262304e+08, 1.391904e+08,
       1.577664e+08]), 'conc': array([7.34908576e-09, 3.39507427e-08, 1.19949935e-07, 3.04395507e-07,
       1.12876743e-06])}}


In [4]:
# evalHGS function goes here

def evalHGS(part, obs):
    # copy direcotry
    dest_dir = './test'
    shutil.copytree('./Base',dest_dir,dirs_exist_ok=True)
    
    # write particle infromat
    lines = '''clear chosen nodes
choose nodes all
initial concentration
0.0d0

!Restart file for concentrations
!./05_initial/IC_sso.cen

clear chosen nodes
Choose nodes block
{0} {1}
{2} {3}
-10 0

Specified mass flux
1
157766400 504921600 {4}

Interpolate mass flux
clear chosen nodes

!----------------------- Time step control
transport time weighting
1.0

concentration control
1e-1

upstream weighting of velocities
1.0 1.0 1.0
!----------------------- Transport properties
use domain type
porous media

clear chosen zones
choose zones all
longitudinal dispersivity
15
transverse dispersivity
2.5
vertical transverse dispersivity
1.5
tortuosity
1.0

    '''.format(part[0]-part[2], part[0]+part[2], part[1]-part[3],part[1]+part[3], part[4])
    
    fname = os.path.join(dest_dir,'01_inc/concentration.inc')
    with open(fname,'w') as fid:
        fid.write(lines)
        
    # Run HGS.
    command = 'cd %s && grok > nul && phgs > nul '%(dest_dir)
    os.system(command)
    
    # evaluate misfit between data.
    weln = [obs[k]['name'] for k in obs]
    
    cost = 0.0
    for i, k in enumerate(obs):
        fname = os.path.join(dest_dir,'*conc.%s.*dat'%(obs[k]['name']))
        fname = glob.glob(fname)[0]
        point = np.loadtxt(fname,skiprows=3)
        # file_list = glob.glob(fname)
        # if not file_list:
        #     print(f"No file found for {obs[k]['name']} in {dest_dir}")
        #     continue
        # fname = file_list[0]

        # searching specific time?
        time = point[:,0]
        pos  = np.zeros(np.shape(time),dtype=bool)
        for j, t in enumerate(time):
            if np.any(t == np.array(obs[k]['time'])):
                pos[j] = True

        cost += np.sum((np.array(obs[k]['conc']) - point[pos,1])**2)
    return cost


In [5]:
# simulated annealing algorithm
def simulated_annealing(objective, bounds, n_iterations, step_size, temp):
	# generate an initial point
	best = bounds[:, 0] + np.random.rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
	# evaluate the initial point
	best_eval = objective(best)
	# current working solution
	curr, curr_eval = best, best_eval
	# run the algorithm
	for i in range(n_iterations):
		# take a step
		candidate = curr + np.random.randn(len(bounds)) * step_size
		# ensure candidate is within bounds
		candidate = np.clip(candidate, bounds[:, 0], bounds[:, 1])
		# evaluate candidate point
		candidate_eval = objective(candidate)
		# check for new best solution
		if candidate_eval < best_eval:
			# store new best point
			best, best_eval = candidate, candidate_eval
			# report progress
			print('>%d f(%s) = %.5f' % (i, best, best_eval))
		# difference between candidate and current point evaluation
		diff = candidate_eval - curr_eval
		# calculate temperature for current epoch
		t = temp / float(i + 1)
		# calculate metropolis acceptance criterion
		metropolis = np.exp(-diff / t)
		# check if we should keep the new point
		if diff < 0 or np.random.rand() < metropolis:
			# store the new current point
			curr, curr_eval = candidate, candidate_eval
	return [best, best_eval]


In [6]:

# define range for input
bounds = np.asarray([[174900, 174950], [532750, 532800], [20, 50], [20, 50], [1.0e-4, 1.0]])
# define the total iterations
n_iterations = 10
# define the maximum step size
step_size = 1
# initial temperature
temp = 100
# perform the simulated annealing search
best, score = simulated_annealing(objective, bounds, n_iterations, step_size, temp)
print('Done!')
print('f(%s) = %f' % (best, score))
